In [1]:
# Dependencies
import pandas as pd
import csv
import requests
from pprint import pprint
from scipy.stats import linregress
import matplotlib.pyplot as plt
from sodapy import Socrata
import os
import gmaps
from config import g_key

In [2]:
# Data collection - year 2022 - post-covid

client = Socrata("data.melbourne.vic.gov.au", app_token="PJ7LNUcljkE0xkoj2J0AzMeU4", username="david_salim2006@hotmail.com", password="David!2022")

results_2022 = client.get("b2ak-trbp", year= 2022, limit = 10000000)

results_2022_df = pd.DataFrame.from_records(results_2022)
results_2022_df.to_csv("foottrafficsdata2022.csv")
results_2022_df.head()

,id,date_time,year,month,mdate,day,time,sensor_id,sensor_name,hourly_counts
0,4279409,2022-05-02T12:00:00.000,2022,May,2,Monday,12,53,Collins St (North),1283
1,4279410,2022-05-02T13:00:00.000,2022,May,2,Monday,13,53,Collins St (North),1180
2,4279411,2022-05-02T14:00:00.000,2022,May,2,Monday,14,53,Collins St (North),1006
3,4279412,2022-05-02T15:00:00.000,2022,May,2,Monday,15,53,Collins St (North),837
4,4279413,2022-05-02T16:00:00.000,2022,May,2,Monday,16,53,Collins St (North),772


In [3]:
# Data 2022 cleaning - data types

results_2022_df.dtypes

id               object
date_time        object
year             object
month            object
mdate            object
day              object
time             object
sensor_id        object
sensor_name      object
hourly_counts    object
dtype: object

In [4]:
# Data 2022 cleaning - data types
data_type_dict = {"id":int,"year":int,"mdate":int,"time":int,"hourly_counts":float}
results_2022_df = results_2022_df.astype(data_type_dict)

results_2022_df.dtypes

id                 int32
date_time         object
year               int32
month             object
mdate              int32
day               object
time               int32
sensor_id         object
sensor_name       object
hourly_counts    float64
dtype: object

In [5]:
# Data 2022 groupby
results_2022_df= results_2022_df[(results_2022_df["time"] >= 12) & (results_2022_df["time"] <= 15)]
location = results_2022_df.groupby("sensor_name")
foottrafic_2022 = location["hourly_counts"].mean()

results_2022_groupby_df = pd.DataFrame({"Av foot traffic 2022": foottrafic_2022})
results_2022_groupby_df = results_2022_groupby_df.reset_index()
results_2022_groupby_df.rename(columns={"sensor_name":"location"},inplace=True)
results_2022_groupby_df


,location,Av foot traffic 2022
0,231 Bourke St,642.963576
1,Alfred Place,191.015203
2,Birrarung Marr,177.092593
3,Bourke St - Spencer St (North),642.864238
4,Bourke St - Spencer St (South),135.953390
...,...,...
67,Town Hall (West),2226.022917
68,Victoria Point,85.004967
69,Waterfront City,209.000000
70,Webb Bridge,257.155629


In [6]:
# Getting location coordinate

Location_list = results_2022_groupby_df["location"].tolist()

location = []
latitude = []
longitude = []

for i in Location_list:
    try:
        base_url = "https://maps.googleapis.com/maps/api/geocode/json"
        target_location2 = i + ", Melbourne Australia"
        params = {"address": target_location2, "key": g_key}
        response = requests.get(base_url, params=params)
        location_geo = response.json()
        lat = location_geo["results"][0]["geometry"]["location"]["lat"]
        lng = location_geo["results"][0]["geometry"]["location"]["lng"]
        print(f"{target_location2}: {lat}, {lng}")
        location.append(i)
        latitude.append(lat)
        longitude.append(lng)
    except:
        print("Target location not found")

231 Bourke St, Melbourne Australia: -37.8133992, 144.9668301
Alfred Place, Melbourne Australia: -37.8143404, 144.9701942
Birrarung Marr, Melbourne Australia: -37.8187351, 144.9742624
Bourke St - Spencer St (North), Melbourne Australia: -37.8170606, 144.9533436
Bourke St - Spencer St (South), Melbourne Australia: -37.8171069, 144.9531785
Bourke St Bridge, Melbourne Australia: -37.8159602, 144.9572025
Bourke St-Russell St (West), Melbourne Australia: -37.8128137, 144.968048
Bourke Street Mall (North), Melbourne Australia: -37.8137685, 144.9644379
Bourke Street Mall (South), Melbourne Australia: -37.8137685, 144.9644379
Chinatown-Lt Bourke St (South), Melbourne Australia: -37.812006, 144.9668036
Chinatown-Swanston St (North), Melbourne Australia: -37.8058235, 144.9631138
Collins Place (North), Melbourne Australia: -37.8144674, 144.9733256
Collins Place (South), Melbourne Australia: -37.8182937, 144.956676
Collins St (North), Melbourne Australia: -37.8182937, 144.956676
Elizabeth St - Flin

In [7]:
# create new df for location coordinate

location_dict = {
    "location": location,
    "Lat": latitude,
    "Lon": longitude,
    }
location_data_dict = pd.DataFrame(location_dict)

location_data_dict.head()

,location,Lat,Lon
0,231 Bourke St,-37.813399,144.966830
1,Alfred Place,-37.814340,144.970194
2,Birrarung Marr,-37.818735,144.974262
3,Bourke St - Spencer St (North),-37.817061,144.953344
4,Bourke St - Spencer St (South),-37.817107,144.953179


In [8]:
# merging foot traffic data and coord data

data_summary_2022 = location_data_dict.merge(results_2022_groupby_df,how="left", on="location" )
data_summary_2022.to_csv ("datasummary2022.csv")
data_summary_2022.head(10)

sort_2022 = data_summary_2022.sort_values("Av foot traffic 2022", ascending=False).reset_index()
sort_2022

top_10_2022 = sort_2022.iloc[0:10,:]
top_10_2022


,index,location,Lat,Lon,Av foot traffic 2022
0,19,Flinders La-Swanston St (West),-37.816485,144.966989,2419.240066
1,67,Town Hall (West),-37.808904,144.929062,2226.022917
2,55,Southbank,-37.823840,144.962497,1819.216887
3,7,Bourke Street Mall (North),-37.813769,144.964438,1812.238411
4,45,Princes Bridge,-37.819240,144.968346,1527.630795
5,39,Melbourne Central,-37.812365,144.962338,1521.129139
6,61,State Library - New,-37.809809,144.965190,1445.673841
7,14,Elizabeth St - Flinders St (East) - New footpath,-37.818148,144.964871,1422.880795
8,40,Melbourne Central-Elizabeth St (East),-37.810236,144.962765,1342.216887
9,41,Melbourne Convention Exhibition Centre,-37.825239,144.951917,1340.178808


In [16]:
# creating google heatmap

# location & foot traffic input
lat_long = top_10_2022[["Lat", "Lon"]]
foottrafic = top_10_2022["Av foot traffic 2022"]

# Plot Heatmap
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_long, weights=foottrafic, max_intensity=2500)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [10]:
# Restaurant nearby

top_resto_name = []
top_resto_address = []
top_resto_lat = []
top_resto_lng = []

for index, row in top_10_2022.iterrows():
    phrase = f"best restaurant in {row.location} Melbourne, Australia"
    target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"

    params = {"query": phrase,"key": g_key, "radius": 100, "type": "restaurant"}

    response = requests.get(target_url, params)
    # print(response.url)
    top_resto = response.json()
    counter = 0
    for place in top_resto["results"]:
        print(place["name"])
        print(place["formatted_address"])
        print(place["geometry"]["location"]["lat"])
        print(place["geometry"]["location"]["lng"])
        top_resto_name.append(place["name"])
        top_resto_address.append(place["formatted_address"])
        top_resto_lat.append(place["geometry"]["location"]["lat"])
        top_resto_lng.append(place["geometry"]["location"]["lng"])
        counter += 1
        if counter > 50:
            break

# phrase = f"best restaurant in Flinders La-Swanston St (West) Melbourne, Australia"
# target_url = "https://maps.googleapis.com/maps/api/place/textsearch/json"
# params = {"query": phrase,"key": g_key}

# response = requests.get(target_url, params)
# top_resto = response.json()
# # pprint(top_resto["results"]["name"])

# pprint(top_resto["results"][0]["geometry"]["location"]["lng"])

# # # pprint(top_resto)


Lollo
408 Flinders Ln, Melbourne VIC 3000, Australia
-37.8182761
144.9603053
Warabi
408 Flinders Ln, Melbourne VIC 3000, Australia
-37.8182761
144.9603053
Roule Galette
Shop 1/241 Flinders Ln, Melbourne VIC 3000, Australia
-37.8168239
144.9662818
Taxi Kitchen
Flinders Street &, Swanston St, Melbourne VIC 3000, Australia
-37.8184283
144.9683284
Hazel
164 Flinders Ln, Melbourne VIC 3000, Australia
-37.8158872
144.9687316
Tian38
350 Flinders Ln, Melbourne VIC 3000, Australia
-37.8176591
144.9627095
Go Hahn
348 Flinders Ln, Melbourne VIC 3000, Australia
-37.8176221
144.9628415
Brunetti Oro Flinders Lane
250 Flinders Ln, Melbourne VIC 3000, Australia
-37.8167025
144.96584
Greta
450 Flinders Ln, Melbourne VIC 3000, Australia
-37.8187583
144.9588661
Saluministi Flinders Lane
388 Flinders Ln, Melbourne VIC 3000, Australia
-37.818121
144.9610715
Culprit
408 Flinders Ln, Melbourne VIC 3000, Australia
-37.8182761
144.9603053
Kisumé
175 Flinders Ln, Melbourne VIC 3000, Australia
-37.8162147
144.96

In [11]:
top_resto_dict = {
    "name": top_resto_name,
    "address": top_resto_address, 
    "Lat": top_resto_lat,
    "Lon": top_resto_lng,
    }
top_resto_df = pd.DataFrame(top_resto_dict)

top_resto_df
len(top_resto_df)


200

In [18]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{name}</dd>
<dt>Address</dt><dd>{address}</dd>
</dl>
"""

resto_info = [info_box_template.format(**row) for index, row in top_resto_df.iterrows()]
locations = top_resto_df[["Lat", "Lon"]]

# Add marker layer ontop of heat map
marker_layer=gmaps.marker_layer(locations, info_box_content=resto_info)
fig.add_layer(marker_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))

In [13]:
# Data for 2019, 2020, 2021
# 7:00 - 10:00 am
# 12:00 - 15:00 pm
# 18:00 - 21:00 pm
# summing hourly foottrafic

In [14]:
# get nearby restaurants
# add markers to identify nearby resto and advice on competition

In [15]:
# correlation
# foottrafic n eatery